In [1]:
from arcgis.gis import GIS
from arcgis.features import Table
from IPython.display import display
from datetime import datetime
gis = GIS()
#Medidas Sanitarias:0b944d9bf1954c71a7fae96bdddee464
Medidas=gis.content.get('0b944d9bf1954c71a7fae96bdddee464')
Medidas

<Item title:"COVID-19 Chile: Medidas Sanitarias" type:Feature Layer Collection owner:lzamudio@soporta.cl_soportachile>